# CityBikes

Send a request to CityBikes for the city of your choice. 

In [1]:
import requests
import pandas as pd
from typing import List, Dict

def get_city_bikes_data(city: str) -> pd.DataFrame:
    """
    Retrieves bike station data from CityBikes API and returns it as a DataFrame.
    
    Args:
        city (str): Name of the city to query (case-insensitive).
    
    Returns:
        pd.DataFrame: DataFrame containing station data with columns:
            - name: Station name
            - latitude: Station latitude
            - longitude: Station longitude
            - free_bikes: Number of available bikes
            - empty_slots: Number of empty slots available.
    """
    base_url = "http://api.citybik.es/v2/networks"
    response = requests.get(base_url)

    if response.status_code != 200:
        raise Exception("Failed to fetch data from CityBikes API")

    data = response.json()

    city_network = next(
        (network for network in data['networks'] 
         if 'location' in network 
         and 'city' in network['location'] 
         and city.lower() in network['location']['city'].lower()),
        None
    )

    if not city_network:
        raise Exception(f"{city} network not found in the API")

    city_network_id = city_network['id']
    city_url = f"{base_url}/{city_network_id}"
    city_response = requests.get(city_url)
    
    if city_response.status_code != 200:
        raise Exception(f"Failed to fetch {city} network data")

    city_data = city_response.json()
    stations = city_data['network']['stations']

    df = pd.DataFrame(stations)
    columns = ['id', 'name', 'latitude', 'longitude', 'free_bikes', 'empty_slots']
    
    missing_columns = [col for col in columns if col not in df.columns]
    if missing_columns:
        raise ValueError(f"Missing required columns: {missing_columns}")

    return df[columns]




Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [2]:
# Example usage
try:
    city_name = input("Hamilton: ").strip()
    hamilton_df = get_city_bikes_data(city_name)
    print(f"{city_name} Bike Stations")
    print(hamilton_df.head())
except Exception as e:
    print(f"Error retrieving bike stations: {e}")

 Bike Stations
                                 id                    name   latitude  \
0  004f9aea9391c0a3197981c85926fdb1            AUH - Marasy  24.451202   
1  02757a31c3a5fd002b1e5cc8e8dc756d          AUH - Al Zeina  24.456953   
2  08ad3c8c91a7dd78ba3532346a592fa6         AUH - Yas Plaza  24.465954   
3  0c6a4152798197fe27137b04498cd1c5           AUH - ADIA HQ  24.486103   
4  258198fc7f167bbf380204595e9375e1  AUH - Al Muneera North  24.452079   

   longitude  free_bikes  empty_slots  
0  54.334510           2            3  
1  54.615754           2           12  
2  54.598305           2            4  
3  54.350087           1            9  
4  54.604651           0            2  


Put your parsed results into a DataFrame.

In [51]:
hamilton_df.head()

,name,latitude,longitude,free_bikes,empty_slots
0,Bay at Strachan,43.267859,-79.867923,8,16
1,MAC Rack ITB,43.259178,-79.920503,2,12
2,Augusta at John,43.251632,-79.868980,7,3
3,Hunter GO Centre,43.253330,-79.869533,4,13
4,Maple at Rothsay - ERI08,43.242163,-79.825003,2,7
